In [14]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [7]:
model = GPT2LMHeadModel.from_pretrained("out_rlhf").eval().to("cuda")

In [8]:
dataset = load_dataset("CarperAI/openai_summarize_tldr", split="test")

Found cached dataset parquet (/data/home/tcbegley/.cache/huggingface/datasets/CarperAI___parquet/CarperAI--openai_summarize_tldr-536d9955f5e6f921/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [15]:
generation_config = GenerationConfig(
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

In [21]:
example = dataset[500]
tokenized_prompt = tokenizer(example["prompt"], return_tensors="pt").to("cuda")

res = model.generate(
    **tokenized_prompt,
    generation_config=generation_config,
)

string_to_write = (
    "Query: \n"
    f"{example['prompt']}\n"
    f"Generated: \n"
    f"{tokenizer.decode(res[0, len(tokenized_prompt['input_ids'][0]):].cpu().numpy().tolist())}\n"
    f"True label: \n"
    f"{example['label']}\n"
    f"====================================================\n"
)
print(string_to_write)

Query: 
SUBREDDIT: r/AskReddit
TITLE: Where's a good place to apply for summer jobs?
POST: Looking ahead to the summer I'm trying to figure out where I can get a job. I'm a 20 year-old university student currently based in the uk, and I love summer, sun, beaches etc. so am trying to find a job that will be near or involve those things. I've looked up a bunch of hotels in bora bora and Greece and will contact them asking if they have any employment opportunities, but are there other locations/hotels/jobs I should be looking at that will likewise allow me to enjoy the best that summer has to offer during my off-time? I am a hard worker, so I'm not simply looking for a lazy summer job, just wanting to maybe travel somewhere where I can enjoy myself a lot in the summer.
TL;DR: 
Generated: 
 Looking for a summer job in the uk, but don't know where to go. I'm looking for a job that will allow me to enjoy the best that summer has to offer during my off-time.<|endoftext|>
True label: 
what's a